In [1]:
import requests
import json
import time
import datetime
import random
import os


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col,regexp_replace, split
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql.functions import monotonically_increasing_id

# Create a Spark session
spark = SparkSession.builder.appName("Load JSON Data").getOrCreate()

# Path to your JSON file
test_json = "./datadump/game.json"
json_file_path = "./datadump/games_rm_1v1_s6.json"

# spark.read.json(spark.sparkContext.wholeTextFiles(json_file_path).values).show()
df = spark.read.json(json_file_path)
df.show(10)
# print(df)
# Creating separate columns for each team object within the teams_flat array
# print(df)?s

+---------------+--------+--------------------+--------+------+-------------+-------+---------+-------+--------------------+--------------------+
|_corrupt_record|duration|         finished_at| game_id|  kind|          map| map_id|    patch| server|          started_at|               teams|
+---------------+--------+--------------------+--------+------+-------------+-------+---------+-------+--------------------+--------------------+
|              [|    NULL|                NULL|    NULL|  NULL|         NULL|   NULL|     NULL|   NULL|                NULL|                NULL|
|           NULL|    1577|2023-11-15T18:33:...|96043210|rm_1v1|   Dry Arabia| 163361|9.1.176.0|     UK|2023-11-15T18:06:...|[[{rus, false, ke...|
|           NULL|     913|2023-11-15T18:22:...|96043197|rm_1v1|Hidden Valley|2144600|9.1.176.0|     UK|2023-11-15T18:06:...|[[{mongols, false...|
|           NULL|    1375|2023-11-15T18:30:...|96043326|rm_1v1|        Gorge|2135363|9.1.176.0|     UK|2023-11-15T18:07:...|

In [3]:
final_df = df.select(
    col("game_id"),
    col("map"),
    col("kind"),
    col("server"),
    col("patch"),
    col("teams")[0][0].alias("team1"),
    col("teams")[1][0].alias("team2")
)
final_df.show(10,truncate=False)

+--------+-------------+------+-------+---------+-------------------------------------------------------------------------+-----------------------------------------------------------------------+
|game_id |map          |kind  |server |patch    |team1                                                                    |team2                                                                  |
+--------+-------------+------+-------+---------+-------------------------------------------------------------------------+-----------------------------------------------------------------------+
|NULL    |NULL         |NULL  |NULL   |NULL     |NULL                                                                     |NULL                                                                   |
|96043210|Dry Arabia   |rm_1v1|UK     |9.1.176.0|{rus, false, keyboard, NULL, NULL, 17518566, NULL, NULL, loss}           |{french, false, keyboard, NULL, NULL, 5391878, NULL, NULL, win}        |
|96043197|Hidden Val

In [4]:
print(final_df.select(col('team1')))
team1_df = final_df.select(
    col("team1.civilization").alias("civilization1"),
    col("team1.civilization_randomized").alias("civilization_randomized1"),
    col("team1.input_type").alias("input_type1"),
    col("team1.mmr").alias("mmr1"),
    col("team1.mmr_diff").alias("mmr_diff1"),
    col("team1.profile_id").alias("profile_id1"),
    col("team1.rating").alias("rating1"),
    col("team1.rating_diff").alias("rating_diff1"),
    col("team1.result").alias("result1")
)
team1_df.show(truncate=False)

DataFrame[team1: struct<civilization:string,civilization_randomized:boolean,input_type:string,mmr:bigint,mmr_diff:bigint,profile_id:bigint,rating:bigint,rating_diff:bigint,result:string>]
+-------------------+------------------------+-----------+----+---------+-----------+-------+------------+-------+
|civilization1      |civilization_randomized1|input_type1|mmr1|mmr_diff1|profile_id1|rating1|rating_diff1|result1|
+-------------------+------------------------+-----------+----+---------+-----------+-------+------------+-------+
|NULL               |NULL                    |NULL       |NULL|NULL     |NULL       |NULL   |NULL        |NULL   |
|rus                |false                   |keyboard   |NULL|NULL     |17518566   |NULL   |NULL        |loss   |
|mongols            |false                   |keyboard   |873 |NULL     |5149239    |NULL   |NULL        |loss   |
|delhi_sultanate    |false                   |keyboard   |1073|17       |15401344   |NULL   |NULL        |win    |
|holy_r

In [5]:
team2_df = final_df.select(
    col("team2.civilization").alias("civilization2"),
    col("team2.civilization_randomized").alias("civilization_randomized2"),
    col("team2.input_type").alias("input_type2"),
    col("team2.mmr").alias("mmr2"),
    col("team2.mmr_diff").alias("mmr_diff2"),
    col("team2.profile_id").alias("profile_id2"),
    col("team2.rating").alias("rating2"),
    col("team2.rating_diff").alias("rating_diff2"),
    col("team2.result").alias("result2")
)
team2_df.show(truncate=False)

+-------------------+------------------------+-----------+----+---------+-----------+-------+------------+-------+
|civilization2      |civilization_randomized2|input_type2|mmr2|mmr_diff2|profile_id2|rating2|rating_diff2|result2|
+-------------------+------------------------+-----------+----+---------+-----------+-------+------------+-------+
|NULL               |NULL                    |NULL       |NULL|NULL     |NULL       |NULL   |NULL        |NULL   |
|french             |false                   |keyboard   |NULL|NULL     |5391878    |NULL   |NULL        |win    |
|french             |false                   |keyboard   |961 |12       |16883606   |NULL   |NULL        |win    |
|malians            |true                    |NULL       |1104|-17      |9825729    |NULL   |NULL        |loss   |
|japanese           |false                   |keyboard   |1161|14       |8695892    |NULL   |NULL        |win    |
|zhu_xis_legacy     |false                   |keyboard   |1066|-16      |8388565

In [6]:
final_df_indexed = final_df.withColumn("index", monotonically_increasing_id())
team1_df_indexed = team1_df.withColumn("index", monotonically_increasing_id())
team2_df_indexed = team2_df.withColumn("index", monotonically_increasing_id())

# Join the DataFrames on the index column
# joined_df = final_df_indexed.join(team1_df_indexed, "index", "inner")

# # Optionally, you can drop the index column if it's no longer needed
# # final_df = joined_df.drop("index")

# # Show the result to verify
# joined_df.show(truncate=False)

In [7]:
final_df_indexed.show(truncate=False)    

+--------+-------------+------+-------+---------+---------------------------------------------------------------------------+---------------------------------------------------------------------------+-----+
|game_id |map          |kind  |server |patch    |team1                                                                      |team2                                                                      |index|
+--------+-------------+------+-------+---------+---------------------------------------------------------------------------+---------------------------------------------------------------------------+-----+
|NULL    |NULL         |NULL  |NULL   |NULL     |NULL                                                                       |NULL                                                                       |0    |
|96043210|Dry Arabia   |rm_1v1|UK     |9.1.176.0|{rus, false, keyboard, NULL, NULL, 17518566, NULL, NULL, loss}             |{french, false, keyboard, NULL, NULL, 53918

In [8]:
team1_df_indexed.show(truncate=False)

+-------------------+------------------------+-----------+----+---------+-----------+-------+------------+-------+-----+
|civilization1      |civilization_randomized1|input_type1|mmr1|mmr_diff1|profile_id1|rating1|rating_diff1|result1|index|
+-------------------+------------------------+-----------+----+---------+-----------+-------+------------+-------+-----+
|NULL               |NULL                    |NULL       |NULL|NULL     |NULL       |NULL   |NULL        |NULL   |0    |
|rus                |false                   |keyboard   |NULL|NULL     |17518566   |NULL   |NULL        |loss   |1    |
|mongols            |false                   |keyboard   |873 |NULL     |5149239    |NULL   |NULL        |loss   |2    |
|delhi_sultanate    |false                   |keyboard   |1073|17       |15401344   |NULL   |NULL        |win    |3    |
|holy_roman_empire  |false                   |keyboard   |1120|-14      |797465     |NULL   |NULL        |loss   |4    |
|japanese           |false      

In [9]:
team2_df_indexed.show(truncate=False)

+-------------------+------------------------+-----------+----+---------+-----------+-------+------------+-------+-----+
|civilization2      |civilization_randomized2|input_type2|mmr2|mmr_diff2|profile_id2|rating2|rating_diff2|result2|index|
+-------------------+------------------------+-----------+----+---------+-----------+-------+------------+-------+-----+
|NULL               |NULL                    |NULL       |NULL|NULL     |NULL       |NULL   |NULL        |NULL   |0    |
|french             |false                   |keyboard   |NULL|NULL     |5391878    |NULL   |NULL        |win    |1    |
|french             |false                   |keyboard   |961 |12       |16883606   |NULL   |NULL        |win    |2    |
|malians            |true                    |NULL       |1104|-17      |9825729    |NULL   |NULL        |loss   |3    |
|japanese           |false                   |keyboard   |1161|14       |8695892    |NULL   |NULL        |win    |4    |
|zhu_xis_legacy     |false      

In [10]:
final_df_indexed = final_df_indexed.join(team1_df_indexed, "index", "inner")
final_df_indexed = final_df_indexed.join(team2_df_indexed, "index", "inner")

# Optionally, you can drop the index column if it's no longer needed
# final_df = joined_df.drop("index")
# Show the result to verify
final_df_indexed.orderBy("index").show(truncate=False)

+-----+--------+-------------+------+-------+---------+---------------------------------------------------------------------------+---------------------------------------------------------------------------+-------------------+------------------------+-----------+----+---------+-----------+-------+------------+-------+-------------------+------------------------+-----------+----+---------+-----------+-------+------------+-------+
|index|game_id |map          |kind  |server |patch    |team1                                                                      |team2                                                                      |civilization1      |civilization_randomized1|input_type1|mmr1|mmr_diff1|profile_id1|rating1|rating_diff1|result1|civilization2      |civilization_randomized2|input_type2|mmr2|mmr_diff2|profile_id2|rating2|rating_diff2|result2|
+-----+--------+-------------+------+-------+---------+---------------------------------------------------------------------------+-